In [ ]:
# google colab

!pip install folium pandas

# Map

This section focuses on the getting of data and creation of the map

https://apex.oracle.com/pls/apex/hackathonjune2024/NilePProject/GpsData?gpsId=1&longi=43.434&lati=434.3443&TimeStamp=2024-06-12 09:43:32&targetOBJ=Boat&gpsOrigin=real

In [4]:

url = 'https://apex.oracle.com/pls/apex/hackathonjune2024/NilePProject/GpsData'
import http.client

connection = http.client.HTTPSConnection(url)
connection.request("GET")
response = connection.getresponse()
print("Status: {} and reason: {}".format(response.status, response.reason))

connection.close()

InvalidURL: nonnumeric port: '//apex.oracle.com/pls/apex/hackathonjune2024/NilePProject/GpsData'

In [8]:
import http.client
import pandas as pd


host = 'apex.oracle.com'
endpoint = '/pls/apex/hackathonjune2024/NilePProject/GpsData'

def convert_json_to_dataframe(json_object):
    return pd.read_json(json_object)


try:
    conn = http.client.HTTPSConnection(host, timeout=10)
    conn.request("GET", endpoint)
    response = conn.getresponse()
    if response.status == 200:
        data = response.read().decode()
        # Verarbeite die JSON-Daten hier weiter
        df = df = convert_json_to_dataframe(data)

    else:
        print(f"Fehler: {response.status} {response.reason}")
    conn.close()
except (ConnectionError, TimeoutError) as e:
    print(f"Fehler beim Verbinden: {e}")


/tmp/ipykernel_15245/2544433438.py:9: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_json(json_object)


ValueError: All arrays must be of the same length

In [1]:
import json
import folium
from folium.plugins import MarkerCluster


In [5]:
fg = folium.FeatureGroup("Lines") # use to plot lines from coordinates on the map

def create_line_color(param: str):
    if param == "boat":
        print("boat")
        # return "#0000ff"
        return "blue"
    if param == "plane":
        print("plane")
        # return "#008000"
        return "green"
    if param == "truck":
        print("truck")
        return "#ff0000"
    if param == "handy":
        return "yellow"


def create_geolocation():
    coords_all = df[["lat", "long"]] # .reset_index(drop=True)
    coords_start = [df.loc[0, "lat"], df.loc[0, "long"]]
    time_start = df.loc[0, "timestamp"]
    coord_last = [df.loc[(len(df) - 1), "lat"], df.loc[(len(df) - 1), "long"]]
    time_last = df.loc[(len(df) - 1), "timestamp"]
    vehicle_last = df.loc[(len(df) - 1), "tracked_object"]

    # (create_line_color(df.loc[index,"tracked_object"]))
        
    basemap = folium.Map(location = coord_last, zoom_start = 13)
    """
    for index, row in coords_all.iterrows():
        lat, long = row["lat"], row["long"]
        print(f"test {lat} {long} and {df.loc[index,"tracked_object"]}")
        folium.PolyLine([(lat,long)],
                        color = "red").add_to(basemap)
    """
    folium.PolyLine(coords_all).add_to(basemap)
    fg.add_to(basemap)
    folium.LayerControl(position='bottomright').add_to(basemap)

    folium.Marker(coords_start,
                  popup = f'Started at: {time_start}',
                  icon=folium.Icon(color='blue')).add_to(basemap)
    folium.Marker(coord_last,
                  popup = f'Last Seen at: {time_last}\nVehicle: {vehicle_last}',
                  icon=folium.Icon(color='red')).add_to(basemap)

    basemap.save("map.html")




df = convert_json_to_dataframe("20240612_162723_tracking_data.json")

create_geolocation()

In [6]:
import webbrowser

webbrowser.open("map.html")

True

# Testing

In [1]:
# testing ground to implement different color lines for each transportation used

In [ ]:
    for geo_point in range(0, len(coords_all)):
        folium.PolyLine(coords_all[geo_point],
                        color = create_line_color(df.iloc[geo_point:"tracked_object"])).add_to(basemap)


In [ ]:
    for geo_point in range(0, len(coords_all)):
        folium.PolyLine(coords_all,
                        color = create_line_color(df.loc[geo_point, "tracked_object"])).add_to(basemap)


In [ ]:
fg = folium.FeatureGroup("Lines") # use to plot lines from coordinates on the map


def create_geolocation():
    coords_all = df[["lat", "long"]].reset_index(drop=True)
    coords_start = [df.loc[0, "lat"], df.loc[0, "long"]]
    time_start = df.loc[0, "timestamp"]
    coord_last = [df.loc[(len(df) - 1), "lat"], df.loc[(len(df) - 1), "long"]]
    time_last = df.loc[(len(df) - 1), "timestamp"]
    vehicle_last = df.loc[(len(df) - 1), "tracked_object"]
        
    basemap = folium.Map(location = coord_last, zoom_start = 13)
    # folium.PolyLine(coord_list).add_to(basemap)
    folium.PolyLine(coords_all).add_to(basemap)
    fg.add_to(basemap)
    folium.LayerControl(position='bottomright').add_to(basemap)

    folium.Marker(coords_start,
                  popup = f'Started at: {time_start}',
                  icon=folium.Icon(color='blue')).add_to(basemap)
    folium.Marker(coord_last,
                  popup = f'Last Seen at: {time_last}\nVehicle: {vehicle_last}',
                  icon=folium.Icon(color='red')).add_to(basemap)

    basemap.save("map.html")

def convert_json_to_dataframe(json_object):
    return pd.read_json(json_object)


df = convert_json_to_dataframe("20240612_162723_tracking_data.json")

create_geolocation()